# 导入包

In [6]:
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei']
import seaborn as sns
from collections import Counter
import pandas as pd
import numpy as np
import json
from copy import deepcopy
import re
import os
def mk_dir(file_path):

    folder = os.path.exists(file_path)
    if not folder:
        os.makedirs(file_path)

In [64]:
file_dir = "Virology"

# 数量

In [2]:
# 英文数字转换程序
import re
_known = {
  'zero': 0,
  'one': 1,
  'a':1,
  'an':1,
  'case':1,
  'two': 2,
  'both':2,
  'three': 3,
  'four': 4,
  'five': 5,
  'six': 6,
  'seven': 7,
  'eight': 8,
  'nine': 9,
  'ten': 10,
  'eleven': 11,
  'twelve': 12,
  'thirteen': 13,
  'fourteen': 14,
  'fifteen': 15,
  'sixteen': 16,
  'seventeen': 17,
  'eighteen': 18,
  'nineteen': 19,
  'twenty': 20,
  'thirty': 30,
  'forty': 40,
  'fifty': 50,
  'sixty': 60,
  'seventy': 70,
  'eighty': 80,
  'ninety': 90
  }
def spoken_word_to_number(n):
  """Assume n is a positive integer".
assert _positive_integer_number('nine hundred') == 900
assert spoken_word_to_number('one hundred') == 100
assert spoken_word_to_number('eleven') == 11
assert spoken_word_to_number('twenty two') == 22
assert spoken_word_to_number('thirty-two') == 32
assert spoken_word_to_number('forty two') == 42
assert spoken_word_to_number('two hundred thirty two') == 232
assert spoken_word_to_number('two thirty two') == 232
assert spoken_word_to_number('nineteen hundred eighty nine') == 1989
assert spoken_word_to_number('nineteen eighty nine') == 1989
assert spoken_word_to_number('one thousand nine hundred and eighty nine') == 1989
assert spoken_word_to_number('nine eighty') == 980
assert spoken_word_to_number('nine two') == 92 # wont be able to convert this one
assert spoken_word_to_number('nine thousand nine hundred') == 9900
assert spoken_word_to_number('one thousand nine hundred one') == 1901
"""
  n = n.lower().strip()
  if n.strip() not in _known:
    output = "Not"
  else:
    if n in _known:
      return _known[n]
    else:
      inputWordArr = re.split('[ -]', n)
    assert len(inputWordArr) > 1 #all single words are known
    #Check the pathological case where hundred is at the end or thousand is at end
    if inputWordArr[-1] == 'hundred':
      inputWordArr.append('zero')
      inputWordArr.append('zero')
    if inputWordArr[-1] == 'thousand':
      inputWordArr.append('zero')
      inputWordArr.append('zero')
      inputWordArr.append('zero')
    if inputWordArr[0] == 'hundred':
      inputWordArr.insert(0, 'one')
    if inputWordArr[0] == 'thousand':
      inputWordArr.insert(0, 'one')
    inputWordArr = [word for word in inputWordArr if word not in ['and', 'minus', 'negative']]
    currentPosition = 'unit'
    prevPosition = None
    output = 0
    for word in reversed(inputWordArr):
      if currentPosition == 'unit':
        number = _known[word]
        output += number
        if number > 9:
          currentPosition = 'hundred'
        else:
          currentPosition = 'ten'
      elif currentPosition == 'ten':
        if word != 'hundred':
          number = _known[word]
          if number < 10:
            output += number*10
          else:
            output += number
        #else: nothing special
        currentPosition = 'hundred'
      elif currentPosition == 'hundred':
        if word not in [ 'hundred', 'thousand']:
          number = _known[word]
          output += number*100
          currentPosition = 'thousand'
        elif word == 'thousand':
          currentPosition = 'thousand'
        else:
          currentPosition = 'hundred'
      elif currentPosition == 'thousand':
        assert word != 'hundred'
        if word != 'thousand':
          number = _known[word]
          output += number*1000
      else:
        assert "Can't be here" == None
  return(output)

In [ ]:
col_ab = "num"
evi_yn = json.load(open("../6.Bert/output/output_%s/%s/nbest_predictions_.json" % (col_ab,file_dir), "r"))
pro_l = {}
txt_l = {}
for i in evi_yn:
    pro = [j["probability"] for j in evi_yn[i]]
    pro_l[i] = np.max(pro)
pro_df = pd.DataFrame(pro_l, index=["rate"]).T
pro_df = pro_df.loc[pro_df["rate"] > 0.1,:]
evi = pd.DataFrame(json.load(open("../6.Bert/output/output_%s/%s/predictions_.json" % (col_ab,file_dir), "r")), \
    index=[col_ab]).T
evi_df = evi.loc[pro_df.index]
# 数字转换
evi_df["%s_origin" % col_ab] = evi_df[col_ab]

for k in evi_df.index:
        
    evi_str = str.lower(evi_df.loc[k, col_ab].strip())
    evi_split = re.split(r'\W', evi_str)
    evi_split = list(set(evi_split)) # 去重
    
    for dig in evi_split:
        
        if dig.isdigit():
            evi_df.loc[k, col_ab] = dig
        else:
            if type(spoken_word_to_number(dig)) is int:
                evi_df.loc[k, col_ab] = str(spoken_word_to_number(dig))

    if evi_df.loc[k, col_ab].isdigit():
        if int(evi_df.loc[k, col_ab]) == 1:
                evi_df.loc[k, col_ab] == "1"
        else:
            if int(evi_df.loc[k, col_ab]) < 10:
                evi_df.loc[k, col_ab] = "1-10"
            else:
                if int(evi_df.loc[k, col_ab]) < 100:
                    evi_df.loc[k, col_ab] = "10-100"
                else:
                    evi_df.loc[k, col_ab] = ">100"
    if evi_df.loc[k, col_ab] not in ["1", "1-10", "10-100", ">100"]:
        # print(evi_df.loc[k, col_ab], evi_str)
        evi_df.loc[k, col_ab] = "none"

evi_df.to_csv("output/%s/%s_df.csv" % (file_dir,col_ab))

In [11]:
np.sum(evi_df["num"] == "none")/evi_df.shape[0]

0.10575

# 年龄

In [16]:
# 英文数字转换程序
import re
_known = {
  'zero': 0,
  'two': 2,
  'three': 3,
  'four': 4,
  'five': 5,
  'six': 6,
  'seven': 7,
  'eight': 8,
  'nine': 9,
  'ten': 10,
  'eleven': 11,
  'twelve': 12,
  'thirteen': 13,
  'fourteen': 14,
  'fifteen': 15,
  'sixteen': 16,
  'seventeen': 17,
  'eighteen': 18,
  'nineteen': 19,
  'twenty': 20,
  'thirty': 30,
  'forty': 40,
  'fifty': 50,
  'sixty': 60,
  'seventy': 70,
  'eighty': 80,
  'ninety': 90
  }
def spoken_word_to_number(n):
  """Assume n is a positive integer".
assert _positive_integer_number('nine hundred') == 900
assert spoken_word_to_number('one hundred') == 100
assert spoken_word_to_number('eleven') == 11
assert spoken_word_to_number('twenty two') == 22
assert spoken_word_to_number('thirty-two') == 32
assert spoken_word_to_number('forty two') == 42
assert spoken_word_to_number('two hundred thirty two') == 232
assert spoken_word_to_number('two thirty two') == 232
assert spoken_word_to_number('nineteen hundred eighty nine') == 1989
assert spoken_word_to_number('nineteen eighty nine') == 1989
assert spoken_word_to_number('one thousand nine hundred and eighty nine') == 1989
assert spoken_word_to_number('nine eighty') == 980
assert spoken_word_to_number('nine two') == 92 # wont be able to convert this one
assert spoken_word_to_number('nine thousand nine hundred') == 9900
assert spoken_word_to_number('one thousand nine hundred one') == 1901
"""
  n = n.lower().strip()
  if n.strip() not in _known:
    output = "Not"
  else:
    if n in _known:
      return _known[n]
    else:
      inputWordArr = re.split('[ -]', n)
    assert len(inputWordArr) > 1 #all single words are known
    #Check the pathological case where hundred is at the end or thousand is at end
    if inputWordArr[-1] == 'hundred':
      inputWordArr.append('zero')
      inputWordArr.append('zero')
    if inputWordArr[-1] == 'thousand':
      inputWordArr.append('zero')
      inputWordArr.append('zero')
      inputWordArr.append('zero')
    if inputWordArr[0] == 'hundred':
      inputWordArr.insert(0, 'one')
    if inputWordArr[0] == 'thousand':
      inputWordArr.insert(0, 'one')
    inputWordArr = [word for word in inputWordArr if word not in ['and', 'minus', 'negative']]
    currentPosition = 'unit'
    prevPosition = None
    output = 0
    for word in reversed(inputWordArr):
      if currentPosition == 'unit':
        number = _known[word]
        output += number
        if number > 9:
          currentPosition = 'hundred'
        else:
          currentPosition = 'ten'
      elif currentPosition == 'ten':
        if word != 'hundred':
          number = _known[word]
          if number < 10:
            output += number*10
          else:
            output += number
        #else: nothing special
        currentPosition = 'hundred'
      elif currentPosition == 'hundred':
        if word not in [ 'hundred', 'thousand']:
          number = _known[word]
          output += number*100
          currentPosition = 'thousand'
        elif word == 'thousand':
          currentPosition = 'thousand'
        else:
          currentPosition = 'hundred'
      elif currentPosition == 'thousand':
        assert word != 'hundred'
        if word != 'thousand':
          number = _known[word]
          output += number*1000
      else:
        assert "Can't be here" == None
  return(output)

In [ ]:
col_ab = "age"
evi_yn = json.load(open("../6.Bert/output/output_%s/%s/nbest_predictions_.json" % (col_ab, file_dir), "r"))
pro_l = {}
txt_l = {}
for i in evi_yn:
    pro = [j["probability"] for j in evi_yn[i]]
    pro_l[i] = np.max(pro)
pro_df = pd.DataFrame(pro_l, index=["rate"]).T
pro_df = pro_df.loc[pro_df["rate"] > 0.1,:]
evi = pd.DataFrame(json.load(open("../6.Bert/output/output_%s/%s/predictions_.json" % (col_ab,file_dir), "r")), \
    index=[col_ab]).T
evi_df = evi.loc[pro_df.index]
# 数字转换
evi_df["%s_origin" % col_ab] = evi_df[col_ab]

for k in evi_df.index:
        
    evi_str = str.lower(evi_df.loc[k, col_ab].strip())
    evi_split = re.split(r'\W', evi_str)

    for dig in evi_split:

        if dig.split(".")[0].isdigit():
            
            if dig.split(".")[0].isdigit() < 100:

                evi_df.loc[k, col_ab] = str(int(dig.split(".")[0]))
             
        else:

            if type(spoken_word_to_number(dig)) is int:

                if spoken_word_to_number(dig) < 100:

                    evi_df.loc[k, col_ab] = str(spoken_word_to_number(dig))
            else:
                if dig in ["infant", "infants", "neonatus", "neonate","neonatal",
                "day", "days", "month", "months", "months"]:
                    evi_df.loc[k, col_ab] = "1岁以下"
                if dig in ["young", "younger", "child", "children","adolescent", "adolescents", "paediatric", "pediatric"]:
                    evi_df.loc[k, col_ab] = "1-17岁"
                if dig in ["adult", "adults", "middle"]:
                    evi_df.loc[k, col_ab] = "17-65岁"
                if dig in ["elder", "elderly", "older", "aging"]:
                    evi_df.loc[k, col_ab] = ">65岁"
        
    if evi_df.loc[k, col_ab].isdigit():

        if int(evi_df.loc[k, col_ab]) < 1:
                evi_df.loc[k, col_ab] = "1岁以下"
        else:
            if int(evi_df.loc[k, col_ab]) < 17:
                evi_df.loc[k, col_ab] = "1-17岁"
            else:
                if int(evi_df.loc[k, col_ab]) < 65:
                    evi_df.loc[k, col_ab] = "17-65岁"
                else:
                    evi_df.loc[k, col_ab] = ">65岁"
        print(evi_df.loc[k, col_ab], evi_split)
    
    if evi_df.loc[k, col_ab] not in ["1岁以下", "1-17岁", "17-65岁", ">65岁"]:
        # print(evi_split)
        evi_df.loc[k, col_ab] = "none"
evi_df.to_csv("output/%s/%s_df.csv" % (file_dir,col_ab))

In [67]:
np.sum(evi_df["age"] == "none")/evi_df.shape[0]

0.315599173553719

# 性别

In [69]:
col_ab = "gender"
evi_yn = json.load(open("../6.Bert/output/output_%s/%s/nbest_predictions_.json" % (col_ab, file_dir), "r"))
txt_l = {}
for i in evi_yn:
    txt0 = " "
    txt = [j["text"] for j in evi_yn[i]]
    txt_l[i] = txt0.join(txt)
pro_df = pd.DataFrame(txt_l, index=[col_ab]).T
evi_df = pro_df

# 添加标签
evi_df["%s_origin" % col_ab] = evi_df[col_ab]

for k in evi_df.index:
        
    evi_str = str.lower(evi_df.loc[k, col_ab].strip())
    evi_split = re.split(r'\W', evi_str)
    evi_split = list(set(evi_split)) # 去重

    for dig in evi_split:
        if dig in ["man", "men", "boy", "boys","male", "males", "he", "gentleman", "gentlemen", "his", "him"]:
            evi_df.loc[k, col_ab] = "男"
        if dig in ["woman", "women", "girl","girls", "female", "females", "fm", "she", "lady", "ladys", "her"]:
            evi_df.loc[k, col_ab] = "女"
    if evi_df.loc[k, col_ab] not in ["男","女"]:
        # print(evi_split)
        evi_df.loc[k, col_ab] = "none"
evi_df.to_csv("output/%s/%s_df.csv" % (file_dir, col_ab))


In [70]:
np.sum(evi_df["gender"] == "none")/evi_df.shape[0]

0.7481120201384519

# 免疫状态

In [71]:
col_ab = "immu"
evi_yn = json.load(open("../6.Bert/output/output_%s/%s/nbest_predictions_.json" % (col_ab, file_dir), "r"))
txt_l = {}
for i in evi_yn:
    txt0 = " "
    txt = [j["text"] for j in evi_yn[i]]
    txt_l[i] = txt0.join(txt)
pro_df = pd.DataFrame(txt_l, index=[col_ab]).T
evi_df = pro_df

# 添加标签
sm_l = ["immunodeficiency", "immunocompetent", "immunocompromised", "immunosuppressive","immunosuppression", 
        "competent", "compromised", "suppressed", "supperssive", "suppression",
        "hiv","aids", "transplantation", "transplanted"
        "malignancy", "cancer", "carcinoma", "leukemia", "lymphoma",
        "ill", "impair", "impaired","obesity", "diabete", "fibrosis"]
evi_df["%s_origin" % col_ab] = evi_df[col_ab]

for k in evi_df.index:
        
    evi_str = str.lower(evi_df.loc[k, col_ab].strip())
    evi_split = re.split(r'\W', evi_str)
    evi_split = list(set(evi_split)) # 去重

    for dig in evi_split:
        if len(dig) >= 3:
            for rm in sm_l:
                if dig in rm:        
                    evi_df.loc[k, col_ab] = "抑制"
                if rm in dig:
                    evi_df.loc[k, col_ab] = "抑制"
    if evi_df.loc[k, col_ab] not in ["抑制"]:
        # print(evi_split)
        evi_df.loc[k, col_ab] = "none"
evi_df.to_csv("output/%s/%s_df.csv" % (file_dir,col_ab))

In [73]:
np.sum(evi_df["immu"] == "none")/evi_df.shape[0]

0.5564820641913153

# 证据等级

In [74]:
col_ab = "evi"
evi_yn = json.load(open("../6.Bert/output/output_%s/%s/nbest_predictions_.json" % (col_ab,file_dir), "r"))
txt_l = {}
for i in evi_yn:
    txt0 = " "
    txt = [j["text"] for j in evi_yn[i]]
    txt_l[i] = txt0.join(txt)
pro_df = pd.DataFrame(txt_l, index=[col_ab]).T
evi_df = pro_df
# 添加标签

evi_df["%s_origin" % col_ab] = evi_df[col_ab]
cla_l = ["1：同时具备肺组织病理+病原学证据",
        "2：肺组织病原学证据",
        "3：血及无菌体液（如脓胸液）",
        "4：气管镜活检组织培养出病原体,或者支气管肺泡灌洗液（BALF）定量培养>10**4",
        "5：气管吸出物或痰/BALF纯培养或优势培养出病原体",
        "6：气管吸出物或痰的分子诊断（PCR、基因组测序等）病原",]
for k in evi_df.index:
        
    evi_str = str.lower(evi_df.loc[k, col_ab].strip())
    evi_split = re.split(r'\W', evi_str)
    evi_split = list(set(evi_split)) # 去重
    for dig in evi_split:
        if dig in ["autopsy", "biopsy", "biopsies", "tissue", "tissues", "histopathology", "histological", "histopathological"]:
            evi_df.loc[k, col_ab] = "1：同时具备肺组织病理+病原学证据"
        if dig in ["pathogen", "pathology", "bacteriology"]:
            evi_df.loc[k, col_ab] = "2：肺组织病原学证据"
        if dig in ["blood","bacteremia", "pleural", "purulent"
                    "serology", "serological", "serum", "molecular", "igg", "igm", "antibodies", "antibody"]:
            evi_df.loc[k, col_ab] = "3：血及无菌体液（如脓胸液）"
        if dig in ["bronchoscopy", "tracheal", "specimen", "specimens", "bal", "bronchoalveolar","lavage"]:
            evi_df.loc[k, col_ab] = "4：气管镜活检组织培养出病原体,或者支气管肺泡灌洗液（BALF）定量培养>10**4"
        if dig in ["aspirate", "aspirates", "aspiration", "secretions", "secretion", "sputum"]:
            evi_df.loc[k, col_ab] ="5：气管吸出物或痰/BALF纯培养或优势培养出病原体"
        if dig in ["pcr","mpcr","rtpcr", "polymerase", "16s", "elisa", "elisas", "nucleic","biochemical", "immunofluorescence","sequencing", 
        "sequence","dna", "rna", "smears", "smears","swabs", "swab"]:
            evi_df.loc[k, col_ab] = "6：气管吸出物或痰的分子诊断（PCR、基因组测序等）病原"
    if evi_df.loc[k, col_ab] not in cla_l:
        # print(k, evi_split)
        evi_df.loc[k, col_ab] = "none"
evi_df.to_csv("output/%s/%s_df.csv" % (file_dir, col_ab))

In [75]:
np.sum(evi_df["evi"] == "none")/evi_df.shape[0]

0.4052863436123348

# 合并机器学习数据

In [ ]:
# 人工标注的
col_ab = "pat"
evi = pd.DataFrame(json.load(open("../6.Bert/output/output_%s/%s/predictions_.json" % (col_ab, file_dir), "r"))).T
pat = evi.loc[evi[0]=="yes", :]
for i in os.listdir("output/%s" % file_dir):
    df = pd.read_csv("output/%s/%s" % (file_dir, i))
    df.index = df.iloc[:, 0]
    na_set =  list(set(pat.index) & set(df.index))

    for j in na_set:
        pat.loc[j, df.columns[1:3]] = df.loc[j, df.columns[1:3]]

path = pat
path["标注方式"] = "模型标注"
path_name  = [ele.split(":")[1].replace("\n", "") for ele in path.index]
path["物种名称"] = path_name
link_l = ["https://pubmed.ncbi.nlm.nih.gov/%s/" % ele.split(":")[0].split("_")[-1] for ele in path.index]
path["原文链接"] = link_l
idx_l =  [ele.split(":")[0] for ele in path.index]
path.index  = idx_l

path.columns = ['是否是致病菌', '预测概率', '年龄', '年龄文本', '证据等级', '证据文本', '性别', '性别文本',
               '免疫状态', '免疫文本', '样本量', '样本量文本', '标注方式', '物种名称', '原文链接']

path = path[['物种名称', '是否是致病菌', '预测概率', '样本量', '样本量文本','年龄', '年龄文本',
           '性别', '性别文本', '免疫状态', '免疫文本', '证据等级', '证据文本', '原文链接', '标注方式']]
path.to_csv("output/%s/bert_label.csv" % file_dir, header=True, index=True)

# 转换为json file

In [81]:
file_dir = "Mycology"

In [82]:
# hum_df = pd.read_csv("human_label.csv", header=0)
md_df = pd.read_csv("output/%s/bert_label.csv" % file_dir, header=0)
# hum_df.columns = md_df.columns
# mach_df = pd.concat((hum_df, md_df), 0)
mach_df= deepcopy(md_df)
mach_df[mach_df.isna()] = "none"
mach_df[mach_df=="NA"] = "none"
mach_df.index = range(mach_df.shape[0])

In [83]:
# 添加年份
input_path = "../2.PubTator/output/%s" % file_dir

# 建立物种大类列表6

sp_l = [ele for ele in os.listdir(input_path) if "_done" not in ele]
sp_l = [ele for ele in sp_l if ".ipynb" not in ele]
id_l = []
year_l = []
for sp in sp_l:

    spec_l = [
        ele for ele in os.listdir("%s/%s" % (input_path, sp))
        if ".ipynb" not in ele
    ]

    for spec in spec_l:

        file_l = [
            ele for ele in os.listdir("%s/%s/%s" % (input_path, sp, spec))
            if ".json" in ele
        ]  # 建立这个物种的三种搜索结果json文件列表

        for file in file_l:

            # 读取json文件中的paper并建立列表
            input_name = "%s/%s/%s/%s" % (input_path, sp, spec, file)
            data_l = json.load(open("%s" % input_name, 'r'))

            for paper in data_l:

                # 读取pubtator的注释
                id = paper["id"]
                id_l.append(id)
                year = paper["year"]
                year_l.append(year)
a = pd.DataFrame((id_l, year_l))
b = a.T.drop_duplicates()
b.columns = ["ID", "Year"]
mach_df["发表年份"] = "none"

for i in range(mach_df.shape[0]):

    if mach_df["Unnamed: 0"][i].split("_")[-1] in id_l:

        mach_df["发表年份"][i]  = year_l[id_l.index(mach_df["Unnamed: 0"][i].split("_")[-1])]

In [85]:
# 数据清洗
mach_dfy = deepcopy(mach_df)
mach_dfy.loc[mach_dfy["样本量"]=="10-100", "样本量"] =  "11-100"
mach_dfy.loc[mach_dfy["样本量"]=="10-101", "样本量"] =  "11-100"
mach_dfy.loc[mach_dfy["样本量"]=="10-102", "样本量"] =  "11-100"
mach_dfy.loc[mach_dfy["样本量"]=="＞100", "样本量"] =  ">100"

mach_dfy.loc[mach_dfy["年龄"]=="1-17", "年龄"] =  "1-17岁"
mach_dfy.loc[mach_dfy["年龄"]=="17-65", "年龄"] =  "18-65岁"
mach_dfy.loc[mach_dfy["年龄"]=="17-65岁", "年龄"] =  "18-65岁"
mach_dfy.loc[mach_dfy["年龄"]=="17-65岁", "年龄"] =  "18-65岁"
mach_dfy.loc[mach_dfy["年龄"]==">65岁", "年龄"] =  "65岁以上"
mach_dfy.loc[mach_dfy["年龄"]=="65以上", "年龄"] =  "65岁以上"


mach_dfy.loc[mach_dfy["免疫状态"]=="损伤", "免疫状态"] =  "抑制"
mach_dfy.loc[mach_dfy["免疫状态"]=="受损", "免疫状态"] =  "抑制"

mach_dfy.loc[mach_dfy["证据等级"]=="环境", "证据等级"] =  "none"
mach_dfy.loc[mach_dfy["证据等级"]=="8：文献报道", "证据等级"] =  "none"
mach_dfy.loc[mach_dfy["证据等级"]=="7：免疫学证据", "证据等级"] =  "3：血及无菌体液（如脓胸液）"
mach_dfy.loc[mach_dfy["证据等级"]=="4：气管镜活检组织培养出病原体,或者支气管肺泡灌洗液（BALF）定量培养>10**4", "证据等级"] =  "4：气管镜活检组织培养出病原体，或者支气管肺泡灌洗液（BALF）定量培养>10**4"
mach_dfy.loc[mach_dfy["是否是致病菌"]=='yes', ['物种名称', '是否是致病菌', '样本量', '年龄', 
           '性别',  '免疫状态', '证据等级',  '原文链接', '标注方式', "发表年份"]].to_excel("output/%s/final_label.xlsx" % file_dir)

In [ ]:
mach_l = []
for i in mach_df.index:
    mach = mach_df.loc[i].values.tolist()
    mach_l.append(mach)

v = []
for i in mach_l:
    if len(i) > 3:
        v.append(i)
    else:
        v.append(i[0])
with open("output/%s/mach_l.json" % file_dir, "w") as f:
    b = json.dumps(v)
    f.write(b)